In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [4]:
dataset = pd.read_csv('credit_simple.csv', sep=';')

dataset.shape

(1000, 8)

In [5]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [7]:
y = dataset['CLASSE']
x = dataset.iloc[:, :-1]
x

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo
...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,29/06/2018,fem div/cas,mobilia/equipamento
996,3857.0,4,40,nenhum,30/06/2018,masculino div/sep,carro usado
997,804.0,4,38,nenhum,03/07/2018,masculino solteiro,radio/tv
998,1845.0,4,23,nenhum,04/07/2019,masculino solteiro,radio/tv


In [10]:
# Quais valores faltantes em x
x.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [12]:
# Preencher os valores nulos com a mediana para o saldo atual
mediana = x['SALDO_ATUAL'].median()
x['SALDO_ATUAL'].fillna(mediana, inplace=True)
mediana

/tmp/ipykernel_33689/2300172079.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x['SALDO_ATUAL'].fillna(mediana, inplace=True)


np.float64(2323.0)

In [13]:
x.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [14]:
# No caso do Estado Civil é categórico, então podemos utilizar moda
agrupado = x.groupby(['ESTADOCIVIL']).size()
agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [16]:
x['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
x.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

In [17]:
# Vamos analisar agora outliers, vamos utilizar 3 desvpad
dp = x['SALDO_ATUAL'].std()
dp


np.float64(685936688.9820067)

In [19]:
x.loc[x['SALDO_ATUAL']>= 2 * dp, 'SALDO_ATUAL']

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [20]:
mediana = x['SALDO_ATUAL'].median()
mediana

np.float64(2323.0)

In [22]:
x.loc[x['SALDO_ATUAL']>= 2 * dp, 'SALDO_ATUAL'] = mediana
x.loc[x['SALDO_ATUAL']>= 2 * dp, 'SALDO_ATUAL']

Series([], Name: SALDO_ATUAL, dtype: float64)

In [23]:
# BINNING
agrupado = x.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [29]:
# classes como qualificação e eletrodomésticos aparecem muito pouco e poderiam
# ser ignoradas na nossa análise, portanto vamos realizar o BINNING
x.loc[x['PROPOSITO']=='Eletrodomésticos', 'PROPOSITO'] = 'outros'
x.loc[x['PROPOSITO']=='qualificação', 'PROPOSITO'] = 'outros'
agrupado = x.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  33
radio/tv               280
dtype: int64

In [30]:
x['DATA']

0      01/01/2019
1      01/01/2020
2      02/01/2020
3      02/01/2019
4      03/01/2018
          ...    
995    29/06/2018
996    30/06/2018
997    03/07/2018
998    04/07/2019
999    05/07/2018
Name: DATA, Length: 1000, dtype: object

In [33]:
# Transformar a data em atributos
x['DATA'] = pd.to_datetime(x['DATA'], format='%d/%m/%Y')
x['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [36]:
x['ANO'] = x['DATA'].dt.year
x['MES'] = x['DATA'].dt.month
x['DIASEMANA'] = x['DATA'].dt.day_name()
x['DIASEMANA']

# Podemos utilizar o labelencoder para transformar essas categorias em valores
# numéricos!

0        Tuesday
1      Wednesday
2       Thursday
3      Wednesday
4      Wednesday
         ...    
995       Friday
996     Saturday
997      Tuesday
998     Thursday
999     Thursday
Name: DIASEMANA, Length: 1000, dtype: object

In [39]:
x['ESTADOCIVIL'].unique()
x['PROPOSITO'].unique()
x['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [41]:
labelencoder1 = LabelEncoder()
x['ESTADOCIVIL'] = labelencoder1.fit_transform(x['ESTADOCIVIL'])
x['PROPOSITO'] = labelencoder1.fit_transform(x['PROPOSITO'])
x['DIASEMANA'] = labelencoder1.fit_transform(x['DIASEMANA']) 

In [ ]:
x.head()

In [ ]:
outros = x['OUTROSPLANOSPGTO'].unique()
outros

In [42]:
z = pd.get_dummies(x['OUTROSPLANOSPGTO'], prefix = 'OUTROS')

In [44]:
sc = StandardScaler()
m = sc.fit_transform(x.iloc[:,0:3])
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]], shape=(1000, 3))

In [45]:
x = pd.concat([x,z,pd.DataFrame(m,columns=['SALDO_ATUAL_N','RESIDENCEADESDE_N','IDADE_N'])],axis=1)
x.head()